![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


Preprocess the Data by:
1. Replacing the missing values
2. Imputing based on Data Type
3. Adding Missing Values with a for loop
4. One-hot Encoding

In [9]:
cc_apps.replace('?', np.nan, inplace=True)

# Impute missing values based on data type
for column in cc_apps.columns:
    if cc_apps[column].dtype == 'object':
        # Fill categorical columns with mode
        cc_apps[column].fillna(cc_apps[column].mode()[0], inplace=True)
    else:
        # Fill numerical columns with mean
        cc_apps[column].fillna(cc_apps[column].mean(), inplace=True)

# One-hot encode categorical variables
cc_apps = pd.get_dummies(cc_apps, drop_first=True)

Preparing the data for modeling
1. Define the target variable
2. Splitting the data
3. Scaling the data

In [10]:
# Define target and feature variables
X = cc_apps.iloc[:, :-1]  # All columns except the last one
y = cc_apps.iloc[:, -1]   # Last column as target

# Ensure all column names are strings
X.columns = X.columns.astype(str)

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2025)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Training the model
1. Instantiating the model. I use Logistic Regression for this purpose.
2. Fitting the model.
3. Generating and evaluating the predictions.

In [11]:
# Instantiate a logistic regression model
model = LogisticRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Generate predictions
y_pred = model.predict(X_test)

# Evaluate the model using a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[38 15]
 [16 69]]


Finding the best scoring model
1. Defining grid search parameters
2. Performing grid search cross validation
3. Find the model with the best score

In [12]:
# Define grid search parameters
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# Perform grid search cross-validation
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Find the best model
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)

Best Parameters: {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}
Best Cross-Validation Score: 0.8604586404586405
